In [3]:
#!/usr/bin/env python
from datetime import datetime
from time import time
from lxml import html,etree
from reviews_final import scrape, write_in_csv
import pandas as pd
import requests,re
import os,sys
import unicodecsv as csv
import argparse

def parse(locality,checkin_date,checkout_date,sort):
    checkIn = checkin_date.strftime("%Y/%m/%d")
    checkOut = checkout_date.strftime("%Y/%m/%d")
    print ("Scraper Inititated for Locality:%s"%locality)
    header = {

                            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
            }
    # TA rendering the autocomplete list using this API
    print ("Finding search result page URL")
    geo_url = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime='+str(int(time()))+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query='+locality
    api_response  = requests.get(geo_url,headers=header, timeout=120).json()
    #getting the TA url for th equery from the autocomplete response
    url_from_autocomplete = "http://www.tripadvisor.com"+api_response['results'][0]['url']
    print ('URL found %s'%url_from_autocomplete)
    geo = api_response['results'][0]['value']
    #Formating date for writing to file
    a=url_from_autocomplete
    b=a.split("-")
    s="-"
    c=s.join([b[0],b[1],"oa30",b[2],b[3]])
    d=s.join([b[0],b[1],"oa60",b[2],b[3]])
    e=s.join([b[0],b[1],"oa90",b[2],b[3]])
    urllist = [a,c,d,e]

    date = checkin_date.strftime("%Y_%m_%d")+"_"+checkout_date.strftime("%Y_%m_%d")
    #form data to get the hotels list from TA for the selected date
    form_data = {'changeSet': 'TRAVEL_INFO',
            'showSnippets': 'false',
            'staydates':date,
            'uguests': '2',
            'sortOrder':sort
        
    }
    
    
    all_hotel=[]
    
    for url_from_autocomplete in urllist:
        print(url_from_autocomplete)

        headers = {
                                'Accept': 'text/javascript, text/html, application/xml, text/xml, */*',
                                'Accept-Encoding': 'gzip,deflate',
                                'Accept-Language': 'en-US,en;q=0.5',
                                'Cache-Control': 'no-cache',
                                'Connection': 'keep-alive',
                                'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
                                'Host': 'www.tripadvisor.com',
                                'Pragma': 'no-cache',
                                'Referer': url_from_autocomplete,
                                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:28.0) Gecko/20100101 Firefox/28.0',
                                'X-Requested-With': 'XMLHttpRequest'
                            }
        cookies=  {"SetCurrency":"USD"}
        print ("Downloading search results page")
        page_response  = requests.post(url = url_from_autocomplete,data=form_data,headers = headers, cookies = cookies, verify=False)
        print ("Parsing results ")
        parser = html.fromstring(page_response.text)
        hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[contains(@class,"listing collapsed")]')
        hotel_data = []
        if not hotel_lists:
            hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[@class="listing "]')

        for hotel in hotel_lists:
            XPATH_HOTEL_LINK = './/a[contains(@class,"property_title")]/@href'
            XPATH_REVIEWS  = './/a[@class="review_count"]//text()'
            XPATH_RANK = './/div[@class="popindex"]//text()'
            XPATH_RATING = './/span[contains(@class,"ui_bubble_rating bubble_45")]/@alt' #update this code to get rating
            XPATH_RATING_2 = './/a[contains(@class,"ui_bubble_rating bubble_45")]/@alt' #update this code to get rating
            XPATH_HOTEL_NAME = './/a[contains(@class,"property_title")]//text()'
            XPATH_HOTEL_FEATURES = './/div[contains(@casls,"common_hotel_icons_list")]//li//text()'
            XPATH_HOTEL_PRICE = './/div[contains(@data-sizegroup,"mini-meta-price")]/text()'
            XPATH_VIEW_DEALS = './/div[contains(@data-ajax-preserve,"viewDeals")]//text()'
            XPATH_BOOKING_PROVIDER = './/div[contains(@data-sizegroup,"mini-meta-provider")]//text()'

            raw_booking_provider = hotel.xpath(XPATH_BOOKING_PROVIDER)
            raw_no_of_deals =  hotel.xpath(XPATH_VIEW_DEALS)
            raw_hotel_link = hotel.xpath(XPATH_HOTEL_LINK)
            raw_no_of_reviews = hotel.xpath(XPATH_REVIEWS)
            raw_rank = hotel.xpath(XPATH_RANK)
            raw_rating = hotel.xpath(XPATH_RATING_2)
            raw_hotel_name = hotel.xpath(XPATH_HOTEL_NAME)
            raw_hotel_features = hotel.xpath(XPATH_HOTEL_FEATURES)
            raw_hotel_price_per_night  = hotel.xpath(XPATH_HOTEL_PRICE)

            url = 'http://www.tripadvisor.com'+raw_hotel_link[0] if raw_hotel_link else  None
            reviews = ''.join(raw_no_of_reviews).replace("reviews","").replace(",","") if raw_no_of_reviews else 0
            rank = ''.join(raw_rank) if raw_rank else None
            rating = ''.join(raw_rating).replace('of 5 bubbles','').strip() if raw_rating else None
            name = ''.join(raw_hotel_name).strip() if raw_hotel_name else None
            hotel_features = ','.join(raw_hotel_features)
            #price_per_night = ''.join(raw_hotel_price_per_night).encode('utf-8').replace('\n','') if raw_hotel_price_per_night else None
            price_per_night = ''.join(raw_hotel_price_per_night).replace('\n','') if raw_hotel_price_per_night else None

            no_of_deals = re.findall("all\s+?(\d+)\s+?",''.join(raw_no_of_deals))
            booking_provider = ''.join(raw_booking_provider).strip() if raw_booking_provider else None

            if no_of_deals:
                no_of_deals = no_of_deals[0]
            else:
                no_of_deals = 0

            data = {
                        'hotel_name':name,
                        'url':url,
                        'locality':locality,
                        'reviews':reviews,
                        'rank':rank,
                        'tripadvisor_rating':rating,
                        'checkOut':checkOut,
                        'checkIn':checkIn,
                        'hotel_features':hotel_features,
                        'price_per_night':price_per_night,
                        'no_of_deals':no_of_deals,
                        'booking_provider':booking_provider

            }
            hotel_data.append(data)
            all_hotel.append(data)
    #Referrer is necessary to get the correct response from TA if not provided they will redirect to home page
    my_df=pd.DataFrame(all_hotel)
    print(my_df['hotel_name'])
    return all_hotel
   


In [19]:
test="2021/12/01,2021/12/05,distLow,Paris"

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument('checkin_date',help = 'Hotel Check In Date (Format: YYYY/MM/DD')
sortorder_help = """
    available sort orders are :\n
    priceLow - hotels with lowest price,
    distLow : Hotels located near to the search center,
    recommended: highest rated hotels based on traveler reviews,
    popularity :Most popular hotels as chosen by Tipadvisor users
    """
parser.add_argument('checkout_date',help = 'Hotel Chek Out Date (Format: YYYY/MM/DD)')
parser.add_argument('sort',help = sortorder_help,default ='popularity ')
parser.add_argument('locality',help = 'Search Locality')
print(test)
args = parser.parse_args(test.split(','))
print (args)
locality = args.locality
checkin_date = datetime.strptime(args.checkin_date,"%Y/%m/%d")
checkout_date = datetime.strptime(args.checkout_date,"%Y/%m/%d")
sort= args.sort
checkIn = checkin_date.strftime("%Y/%m/%d")
checkOut = checkout_date.strftime("%Y/%m/%d")
today = datetime.now()

2021/12/01,2021/12/05,distLow,Paris
Namespace(checkin_date='2021/12/01', checkout_date='2021/12/05', sort='distLow', locality='Paris')


In [21]:
data = parse(locality,checkin_date,checkout_date,sort)

Scraper Inititated for Locality:Paris
Finding search result page URL
URL found http://www.tripadvisor.com/Hotels-g187147-Paris_Ile_de_France-Hotels.html
http://www.tripadvisor.com/Hotels-g187147-Paris_Ile_de_France-Hotels.html


/opt/homebrew/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
http://www.tripadvisor.com/Hotels-g187147-oa30-Paris_Ile_de_France-Hotels.html


/opt/homebrew/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
http://www.tripadvisor.com/Hotels-g187147-oa60-Paris_Ile_de_France-Hotels.html


/opt/homebrew/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
http://www.tripadvisor.com/Hotels-g187147-oa90-Paris_Ile_de_France-Hotels.html


/opt/homebrew/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
0                  Hotel Belfast
1          Hotel Malte - Astotel
2            Hotel 34B - Astotel
3         Hotel George - Astotel
4        Hotel de Londres Eiffel
                 ...            
122           Hotel le Chat Noir
123       Hotel Providence Paris
124                   Tiquetonne
125          Grand Hotel Chicago
126    Hotel Louvre Bons Enfants
Name: hotel_name, Length: 127, dtype: object


In [22]:
len(data)

127

In [25]:
my_df = pd.DataFrame(data)

In [26]:
my_df.head(15)

,hotel_name,url,locality,reviews,rank,tripadvisor_rating,checkOut,checkIn,hotel_features,price_per_night,no_of_deals,booking_provider
0,Hotel Belfast,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,198,None,None,2021/12/05,2021/12/01,,$136,14,Hotels.com
1,Hotel Malte - Astotel,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,1657,"#1 Best Value of 2,490 places to stay in Paris",None,2021/12/05,2021/12/01,,$141,19,Hotels.com
2,Hotel 34B - Astotel,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,2775,"#2 Best Value of 2,490 places to stay in Paris",None,2021/12/05,2021/12/01,,$127,18,Hotels.com
3,Hotel George - Astotel,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,1520,"#3 Best Value of 2,490 places to stay in Paris",4.5,2021/12/05,2021/12/01,,$101,15,Official Site
4,Hotel de Londres Eiffel,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,1792,"#4 Best Value of 2,490 places to stay in Paris",4.5,2021/12/05,2021/12/01,,$155,0,Official Site
5,Hotel la Nouvelle Republique,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,627,"#5 Best Value of 2,490 places to stay in Paris",None,2021/12/05,2021/12/01,,$112,14,Booking.com
6,Hotel des Trois Colleges,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,223,None,None,2021/12/05,2021/12/01,,$157,15,Booking.com
7,Hotel Moderne Saint Germain,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,249,"#6 Best Value of 2,490 places to stay in Paris",4.5,2021/12/05,2021/12/01,,$102,15,Booking.com
8,Hotel Raphael,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,886,"#7 Best Value of 2,490 places to stay in Paris",4.5,2021/12/05,2021/12/01,,$425,20,Hotels.com
9,Hotel Madison,http://www.tripadvisor.com/Hotel_Review-g18714...,Paris,1283,"#8 Best Value of 2,490 places to stay in Paris",4.5,2021/12/05,2021/12/01,,$287,18,Booking.com


In [27]:
print(my_df['hotel_name'])

0                  Hotel Belfast
1          Hotel Malte - Astotel
2            Hotel 34B - Astotel
3         Hotel George - Astotel
4        Hotel de Londres Eiffel
                 ...            
122           Hotel le Chat Noir
123       Hotel Providence Paris
124                   Tiquetonne
125          Grand Hotel Chicago
126    Hotel Louvre Bons Enfants
Name: hotel_name, Length: 127, dtype: object


In [28]:
for i in data:
    for y in i:
        print (y,':',i[y])

hotel_name : Hotel Belfast
url : http://www.tripadvisor.com/Hotel_Review-g187147-d593228-Reviews-Hotel_Belfast-Paris_Ile_de_France.html
locality : Paris
reviews : 198 
rank : None
tripadvisor_rating : None
checkOut : 2021/12/05
checkIn : 2021/12/01
hotel_features : 
price_per_night : $136
no_of_deals : 14
booking_provider : Hotels.com
hotel_name : Hotel Malte - Astotel
url : http://www.tripadvisor.com/Hotel_Review-g187147-d228694-Reviews-Hotel_Malte_Astotel-Paris_Ile_de_France.html
locality : Paris
reviews : 1657 
rank : #1 Best Value of 2,490 places to stay in Paris
tripadvisor_rating : None
checkOut : 2021/12/05
checkIn : 2021/12/01
hotel_features : 
price_per_night : $141
no_of_deals : 19
booking_provider : Hotels.com
hotel_name : Hotel 34B - Astotel
url : http://www.tripadvisor.com/Hotel_Review-g187147-d9566832-Reviews-Hotel_34B_Astotel-Paris_Ile_de_France.html
locality : Paris
reviews : 2775 
rank : #2 Best Value of 2,490 places to stay in Paris
tripadvisor_rating : None
checkOut 

In [39]:
import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    # num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    # num_reviews = num_reviews[1:-1] 
    # num_reviews = num_reviews.replace(',', '')
    # num_reviews = int(num_reviews) # convert text into integer
    # print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

    while(True):
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 5:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]

        item = {
            'review_body': review.find('p', class_='partial_entry').text,
            'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
        }

        items.append(item)
        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)

    print()

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        

In [ ]:
DB_COLUMN   = 'review_body'
DB_COLUMN1 = 'review_date'

start_urls = [
    'https://www.tripadvisor.com/Hotel_Review-g187147-d650135-Reviews-Le_Relais_des_Halles-Paris_Ile_de_France.html',
]

print(start_urls)
lang = 'en'

headers = [ 
    DB_COLUMN, 
    DB_COLUMN1, 
]

for url in start_urls:

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)

    if not items:
        print('No reviews')
    else:
        # write in CSV
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')

In [ ]:
https://www.tripadvisor.com/Hotel_Review-g187147-d650135-Reviews-Le_Relais_des_Halles-Paris_Ile_de_France.html
https://www.tripadvisor.com/Hotel_Review-g187147-d650135-Reviews-or0-Le_Relais_des_Halles-Paris_Ile_de_France.html

this part below works

In [142]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import requests
from bs4 import BeautifulSoup as soup


# In[2]:

header = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
html = requests.get('https://www.tripadvisor.in/Hotels-g187147-Paris_Ile_de_France-Hotels.html',headers=header)
html2 = requests.get('https://www.tripadvisor.in/Hotels-g187147-oa30-Paris_Ile_de_France-Hotels.html',headers=header)
html3 = requests.get('https://www.tripadvisor.in/Hotels-g187147-oa60-Paris_Ile_de_France-Hotels.html',headers=header)
html.status_code

200

In [143]:



# In[3]:


bsobj = soup(html.content,'lxml')
print('first_step')


# In[4]:


links = []

for review in bsobj.findAll('a',{'class':'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.in'+ a
    b = review['href']
    b = 'https://www.tripadvisor.in'+ b
    c = review['href']
    c = 'https://www.tripadvisor.in'+ c
    d = review['href']
    d = 'https://www.tripadvisor.in'+ d
    e = review['href']
    e = 'https://www.tripadvisor.in'+ e
    links.append('https://www.tripadvisor.in/'+a)
    print(a)
    #hotel_name = a.find('h1', id='HEADING').text
        
    print(a.find('Reviews'))
    a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    b = b[:(b.find('Reviews')+7)] + '-or5' + b[(b.find('Reviews')+7):]
    c = c[:(c.find('Reviews')+7)] + '-or10' + c[(c.find('Reviews')+7):]
    d = d[:(d.find('Reviews')+7)] + '-or15' + d[(d.find('Reviews')+7):]
    e = e[:(e.find('Reviews')+7)] + '-or20' + e[(e.find('Reviews')+7):]

    #print(hotel_name)
    links.append(a)
    links.append(b)
    links.append(c)
    links.append(d)
    links.append(e)
#links


# In[5]:
bsobj = soup(html2.content,'lxml')
print('second_step')


# In[4]:




for review in bsobj.findAll('a',{'class':'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.in'+ a
    b = review['href']
    b = 'https://www.tripadvisor.in'+ b
    c = review['href']
    c = 'https://www.tripadvisor.in'+ c
    d = review['href']
    d = 'https://www.tripadvisor.in'+ d
    e = review['href']
    e = 'https://www.tripadvisor.in'+ e
    links.append('https://www.tripadvisor.in/'+a)
    print(a)
    #hotel_name = a.find('h1', id='HEADING').text
        
    print(a.find('Reviews'))
    a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    b = b[:(b.find('Reviews')+7)] + '-or5' + b[(b.find('Reviews')+7):]
    c = c[:(c.find('Reviews')+7)] + '-or10' + c[(c.find('Reviews')+7):]
    d = d[:(d.find('Reviews')+7)] + '-or15' + d[(d.find('Reviews')+7):]
    e = e[:(e.find('Reviews')+7)] + '-or20' + e[(e.find('Reviews')+7):]

    #print(hotel_name)
    links.append(a)
    links.append(b)
    links.append(c)
    links.append(d)
    links.append(e)

bsobj = soup(html3.content,'lxml')
print('third_step')
for review in bsobj.findAll('a',{'class':'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.in'+ a
    b = review['href']
    b = 'https://www.tripadvisor.in'+ b
    c = review['href']
    c = 'https://www.tripadvisor.in'+ c
    d = review['href']
    d = 'https://www.tripadvisor.in'+ d
    e = review['href']
    e = 'https://www.tripadvisor.in'+ e
    links.append('https://www.tripadvisor.in/'+a)
    print(a)
    #hotel_name = a.find('h1', id='HEADING').text
        
    print(a.find('Reviews'))
    a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    b = b[:(b.find('Reviews')+7)] + '-or5' + b[(b.find('Reviews')+7):]
    c = c[:(c.find('Reviews')+7)] + '-or10' + c[(c.find('Reviews')+7):]
    d = d[:(d.find('Reviews')+7)] + '-or15' + d[(d.find('Reviews')+7):]
    e = e[:(e.find('Reviews')+7)] + '-or20' + e[(e.find('Reviews')+7):]

    #print(hotel_name)
    links.append(a)
    links.append(b)
    links.append(c)
    links.append(d)
    links.append(e)



first_step
https://www.tripadvisor.in/Hotel_Review-g187147-d650135-Reviews-Le_Relais_des_Halles-Paris_Ile_de_France.html#REVIEWS
56
https://www.tripadvisor.in/Hotel_Review-g187147-d234640-Reviews-Hotel_George_Astotel-Paris_Ile_de_France.html#REVIEWS
56
https://www.tripadvisor.in/Hotel_Review-g187147-d197946-Reviews-Hotel_Bradford_Elysees_Astotel-Paris_Ile_de_France.html#REVIEWS
56
https://www.tripadvisor.in/Hotel_Review-g187147-d9566832-Reviews-Hotel_34B_Astotel-Paris_Ile_de_France.html#REVIEWS
57
https://www.tripadvisor.in/Hotel_Review-g187147-d12829774-Reviews-Hotel_la_Nouvelle_Republique-Paris_Ile_de_France.html#REVIEWS
58
https://www.tripadvisor.in/Hotel_Review-g187147-d230431-Reviews-Hotel_Astoria_Astotel-Paris_Ile_de_France.html#REVIEWS
56
https://www.tripadvisor.in/Hotel_Review-g187147-d196829-Reviews-Hotel_Chopin-Paris_Ile_de_France.html#REVIEWS
56
https://www.tripadvisor.in/Hotel_Review-g187147-d265476-Reviews-Hotel_de_Londres_Eiffel-Paris_Ile_de_France.html#REVIEWS
56
https:/

In [145]:
links[0:9]

['https://www.tripadvisor.in/https://www.tripadvisor.in/Hotel_Review-g187147-d650135-Reviews-Le_Relais_des_Halles-Paris_Ile_de_France.html#REVIEWS',
 'https://www.tripadvisor.in/Hotel_Review-g187147-d650135-Reviews-or{}-Le_Relais_des_Halles-Paris_Ile_de_France.html#REVIEWS',
 'https://www.tripadvisor.in/Hotel_Review-g187147-d650135-Reviews-or5-Le_Relais_des_Halles-Paris_Ile_de_France.html#REVIEWS',
 'https://www.tripadvisor.in/Hotel_Review-g187147-d650135-Reviews-or10-Le_Relais_des_Halles-Paris_Ile_de_France.html#REVIEWS',
 'https://www.tripadvisor.in/Hotel_Review-g187147-d650135-Reviews-or15-Le_Relais_des_Halles-Paris_Ile_de_France.html#REVIEWS',
 'https://www.tripadvisor.in/Hotel_Review-g187147-d650135-Reviews-or20-Le_Relais_des_Halles-Paris_Ile_de_France.html#REVIEWS',
 'https://www.tripadvisor.in/https://www.tripadvisor.in/Hotel_Review-g187147-d234640-Reviews-Hotel_George_Astotel-Paris_Ile_de_France.html#REVIEWS',
 'https://www.tripadvisor.in/Hotel_Review-g187147-d234640-Reviews-or

In [129]:

from random import randint
from time import sleep

reviews = []
all = []

for link in links:
    d = [5,10,15,20,25]
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
    html2 = requests.get(link.format(i for i in range(5,1000,5)),headers=headers)
    sleep(randint(1,5))
    bsobj2 = soup(html2.content,'lxml')
    hotel_name = bsobj2.find('h1', id='HEADING').text
    #print(hotel_name)
    
    for r in bsobj2.findAll('q'):
        reviews.append(r.span.text.strip())
        #print(link)
        #print(r.span.text.strip())
        all.append([hotel_name,link,r.span.text.strip()])
        
#reviews


In [130]:
combined = pd.DataFrame(all,columns=['Hotel','link','review'])

In [140]:
len(combined['Hotel'].drop_duplicates())

60

In [141]:
combined['Hotel'].value_counts()

Hotel Malte - Astotel                       30
Hotel George - Astotel                      30
Hotel Signature St Germain des Pres         30
Les Jardins de Mademoiselle                 30
Hotel La Manufacture                        30
Hotel Grand Powers                          30
Le Pavillon de la Reine                     30
Fauchon L'Hotel Paris                       30
Hotel R de Paris                            30
La Chambre du Marais                        30
Le Relais du Louvre                         30
25hours Hotel Terminus Nord                 30
Novotel Paris Centre Gare Montparnasse      30
Hotel Le Mareuil                            30
Le Relais Saint Charles                     30
Pley Hotel                                  30
Palym Hotel                                 30
Minerve Hotel                               30
Cler Hotel                                  30
Hotel Elysees Union                         30
Warwick Paris                               30
Hotel Mademoi

In [122]:
combined.to_csv("combined_paris.csv",index=False)

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Paris"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Chicago"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "New York"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "San Francisco"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Istanbul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Singapore"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bombay"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bangkok"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Sydney"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "London"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Rome"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Athens"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Miami"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Austin"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Tokyo"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bali"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Osaka"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Hong Kong"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Dubai"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Phuket"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Seoul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Prague"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Milan"

In [ ]:
test="2020/11/01,2020/11/05,popularity,Milan"

In [ ]:
test.split(',')

In [ ]:
args
print(locality)

In [ ]:
if today<datetime.strptime(checkIn,"%Y/%m/%d") and datetime.strptime(checkIn,"%Y/%m/%d")<datetime.strptime(checkOut,"%Y/%m/%d"):
        data = parse(locality,checkin_date,checkout_date,sort)
        print ("Writing to output file tripadvisor_data.csv")
        with open('tripadvisor_data.csv','wb')as csvfile:
            fieldnames = ['hotel_name','url','locality','reviews','tripadvisor_rating','checkIn','checkOut','price_per_night','booking_provider','no_of_deals','hotel_features']
            my_df  = pd.DataFrame(columns = fieldnames)
            for row in  data:
                my_df.loc[len(my_df)] = [row['hotel_name'],row['url'],row['locality'],row['reviews'],row['tripadvisor_rating'],row['checkIn'],row['checkOut'],row['price_per_night'],row['booking_provider'],row['no_of_deals'],row['hotel_features']]
            DB_COLUMN   = 'review_body'
            DB_COLUMN1 = 'review_date'
            DB_COLUMN2 = 'hotelName'
            DB_COLUMN3 = 'hotelUrl'
            start_urls = []
            for index, row2 in my_df.iterrows():
                start_urls.append(row2['url'])
            lang = 'en'
    
            headers = [
            DB_COLUMN,
            DB_COLUMN1,
            DB_COLUMN2,
            DB_COLUMN3
            ]
            reviews_df  = pd.DataFrame(columns = headers)
            
            for url in start_urls:

                # get all reviews for 'url' and 'lang'
                items = scrape(url, lang)
                filename = 'hotelReviewsIn' + locality + '__' + lang
                filename2 = "HotelListIn" + locality + '__' + lang
                if not items:
                    print('No reviews')
                else:
                    # write in CSV
                    for i in items:
                        reviews_df.loc[len(reviews_df)] = [i['review_body'],i['review_date'],my_df.loc[my_df['url'] == url]['hotel_name'], url]
            ci = checkIn.replace("/", "")
            co = checkOut.replace("/", "")
            reviews_df.to_csv(filename + ci +co+'.csv', encoding='utf-8')     
            my_df.to_csv(filename2 + ci +co+'.csv', encoding='utf-8')       
    #checking whether the entered date is already passed
elif today>datetime.strptime(checkIn,"%Y/%m/%d") or today>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: Please enter a valid checkin and checkout dates,entered date is already passed")

elif datetime.strptime(checkIn,"%Y/%m/%d")>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: CheckIn date must be less than checkOut date")


In [6]:
data = parse(locality,checkin_date,checkout_date,sort)

NameError: name 'locality' is not defined

In [ ]:
len(data)

In [ ]:
http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa60-Milan_Lombardy-Hotels.html

In [ ]:
a="http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html"

In [ ]:
b=a.split("-")

In [ ]:
print (b)

In [ ]:
print(b[0])

In [ ]:
s="-"

In [ ]:
c=s.join([b[0],b[1],"oa30",b[2],b[3]])
d=s.join([b[0],b[1],"oa60",b[2],b[3]])

In [ ]:
urllist = [a,c,d]

In [ ]:
urllist